# TesorFlow 基础


TensorFlow 是一个面向深度学习算法的科学计算库，内部数据保存在张量(Tensor)对象上，所有的运算操作(Operation，简称 OP)也都是基于张量对象进行的。复杂的神经网络算法本质上就是各种张量相乘、相加等基本运算操作的组合

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, datasets, preprocessing

gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except RuntimeError as e:
    print(e)

## 1 数据类型
### 1.1数值类型

数值类型的张量是 TensorFlow 的主要数据载体，根据维度数来区分，可分为：
- 标量(scalar): 单个的实数，如 1.2, 3.4 等，维度(Dimension)数为 0，shape 为[]
- 向量(vector): 𝑛个实数的有序集合，通过中括号包裹，如\[1.2\]，\[1.2, 3.4\]等，维度数为1，长度不定，shape为\[n\]
- 张量(Tensor): 所有维度数dim > 2的数组统称为张量. 张量的每个维度也作轴(axis), 一般维度代表了具体的物理含义.
  \[2, 32, 32, 3\] -> 2张图片,高宽均为32,有RGB3个通道

在 TensorFlow 中间，为了表达方便，一般把标量、向量、矩阵也统称为张量，不作区分，需要根据张量的维度数或形状自行判断

In [2]:
# 标量
a = 2.4
aa = tf.constant(1.2)  # 需要使用tf的方式创建张量, 否则不能使用tf的功能函数
type(a), type(aa), tf.is_tensor(aa)

(float, tensorflow.python.framework.ops.EagerTensor, True)

In [4]:
# 向量
a = tf.constant([1, 2])  # 向量的定义须通过 List 容器传给constant()函数
a

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [5]:
b = tf.constant([1, 2.1])
b

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1. , 2.1], dtype=float32)>

In [6]:
b.numpy()  # 返回numpy类型数据

array([1. , 2.1], dtype=float32)

In [8]:
# 定义矩阵
a = tf.constant([[1, 2], [3, 4]])
a, a.shape

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]], dtype=int32)>,
 TensorShape([2, 2]))

In [9]:
# 3维的张量
a = tf.constant([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
a

<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]], dtype=int32)>

### 1.2 字符串(String)类型


In [10]:
s = tf.constant('Hello, TensorFlow')
s

<tf.Tensor: shape=(), dtype=string, numpy=b'Hello, TensorFlow'>

In [12]:
tf.strings.lower(s), tf.strings.upper(s)

(<tf.Tensor: shape=(), dtype=string, numpy=b'hello, tensorflow'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'HELLO, TENSORFLOW'>)

In [13]:
tf.strings.split(s)

<tf.Tensor: shape=(2,), dtype=string, numpy=array([b'Hello,', b'TensorFlow'], dtype=object)>

### 1.3 布尔(Boolean)类型

In [15]:
a = tf.constant(True)
a

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [21]:
b = tf.constant([True, False])
b

<tf.Tensor: shape=(2,), dtype=bool, numpy=array([ True, False])>

In [22]:
# tensorflow 的bool类型与python的bool类型不等价
a is True

False

In [23]:
a == True  # 数值比较

<tf.Tensor: shape=(), dtype=bool, numpy=True>

## 2 数值精度
对于数值类型的张量，可以保存为不同字节长度的精度,位越长，精度越高，同时占用的内存空间也就越大。常用的精度类型有 tf.int16、tf.int32、tf.int64、tf.float16、tf.float32、tf.float64(tf.double)等，

In [24]:
# 制定数值的精度
a = tf.constant(123456789, dtype=tf.int16)  # -32768 ~ +32767
b = tf.constant(123456789, dtype=tf.int32)
a, b

(<tf.Tensor: shape=(), dtype=int16, numpy=-13035>,
 <tf.Tensor: shape=(), dtype=int32, numpy=123456789>)

In [27]:
import numpy as np

pi_1 = tf.constant(np.pi, dtype=tf.float32)
pi_2 = tf.constant(np.pi, dtype=tf.float64)
pi_1, pi_2

(<tf.Tensor: shape=(), dtype=float32, numpy=3.1415927>,
 <tf.Tensor: shape=(), dtype=float64, numpy=3.141592653589793>)

大部分深度学习算法，一般使用 tf.int32 和 tf.float32 可满足大部分场合的运算精度要求(默认都是int32与float32)

In [28]:
# dtype 成员属性可以判断张量的保存精度

print('before:', pi_1.dtype)
if pi_1.dtype != tf.float64:
    pi_1 = tf.cast(pi_1, tf.float64)  # 转化精度
print('after:', pi_1.dtype)

before: <dtype: 'float32'>
after: <dtype: 'float64'> tf.Tensor(3.1415927410125732, shape=(), dtype=float64)


In [29]:
# 类型转换 
# 低->高
a = tf.constant(np.pi, dtype=tf.float16)
tf.cast(a, tf.double)

<tf.Tensor: shape=(), dtype=float64, numpy=3.140625>

In [30]:
# 高 ->低　溢出风险
a = tf.constant(123456789, tf.int32)
tf.cast(a, tf.int16)

<tf.Tensor: shape=(), dtype=int16, numpy=-13035>

In [33]:
# bool -> int
a = tf.constant([True, False])
tf.cast(a, tf.int32)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 0], dtype=int32)>

In [35]:
# int -> bool
a = tf.constant([1, -1, 0, 2])  # 0 为False 其他皆为True
tf.cast(a, tf.bool)

<tf.Tensor: shape=(4,), dtype=bool, numpy=array([ True,  True, False,  True])>

## 3 待优化的张量
为了区分需要计算梯度信息的张量与不需要计算梯度信息的张量，TensorFlow 增加了一种专门的数据类型来支持梯度信息的记录：tf.Variable. 它在普通张量类型基础上添加了name, trainable等属性来支持计算图的构建.

对于不需要的优化的张量，如神经网络的输入𝑿，不需要通过 tf.Variable 封装；相反，对于需要计算梯度并优化的张量，如神经网络层的𝑾和𝒃，需要通过 tf.Variable 包裹以便 TensorFlow 跟踪相关梯度信息。
通过 tf.Variable()函数可以将普通张量转换为待优化张量，例如：

In [36]:
a = tf.constant([1, -1, 2, 0])
aa = tf.Variable(a)  # 转为Variable类型

In [37]:
aa.name  # 用于命名计算图中的变量, 内部维护

'Variable:0'

In [38]:
aa.trainable  # 表征当前张量是否需要被优化

True

In [39]:
# 直接创建tf.Variable类型
a = tf.Variable([[1, 2], [3, 4]])
a

<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

待优化张量可视为普通张量的特殊类型，普通张量其实也可以通过`GradientTape.watch()`方法临时加入跟踪梯度信息的列表，从而支持自动求导功能

## 4 创建张量


In [40]:
# 从数组 列表 创建
tf.convert_to_tensor([1, 2.])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [41]:
# 从numpy数组中创建

tf.convert_to_tensor(np.array([[1, 2.], [3, 4.]]))  # numpy 浮点型默认float64

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[1., 2.],
       [3., 4.]])>

In [42]:
# 全零向量
tf.zeros([3])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [44]:
# 全1向量
tf.ones([2, 2, 3])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

In [45]:
a = tf.constant([[1, 2], [3, 4]])
tf.zeros_like(a)  # 与某个张量一致

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[0, 0],
       [0, 0]], dtype=int32)>

In [46]:
tf.ones_like(a)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1]], dtype=int32)>

In [47]:
# 自定义数值的张量

tf.fill([2, 2], 11)  # 2行2列 数值都为11

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[11, 11],
       [11, 11]], dtype=int32)>

In [48]:
# 指定概率分布
# 正态分布 均值0 标准差1 shape: (2, 3)
tf.random.normal([2, 3], mean=0.0, stddev=1.0)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.84015054,  1.5306568 ,  1.2274808 ],
       [ 0.7496108 , -0.93318796,  0.24387702]], dtype=float32)>

In [49]:
# 采样自[0, 1]的均匀分布

tf.random.uniform([6])

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([0.94168234, 0.6843318 , 0.9218265 , 0.27892482, 0.94699883,
       0.78494585], dtype=float32)>

In [50]:
# 采样自区间[0,10)，shape 为[2,2]的矩阵
tf.random.uniform([2, 2], maxval=10)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[7.577312, 6.646731],
       [5.51612 , 4.221629]], dtype=float32)>

In [56]:
# 类型为整数
tf.random.uniform([2, 2], maxval=100, dtype=tf.int32)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[50, 76],
       [55, 36]], dtype=int32)>

In [58]:
# 创建序列
tf.range(10)  # np.arange(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [59]:
tf.range(10, delta=2)  # 指定步长

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 2, 4, 6, 8], dtype=int32)>

## 5 张量的典型应用
---
### 标量
标量是简单的数字, 维度数为0, shape为[].典型用途是表示各种`误差值`, `测量指标`, 如精确度(Accuracy, acc), 查准度(Precisoin)和查全率(Recall).


In [60]:
out = tf.random.uniform([4, 10])  # 随机模拟网络输出
y = tf.constant([2, 3, 2, 0])  # 随机构造样本真实标签
y = tf.one_hot(y, depth=10)  # one-hot编码处理
loss = tf.keras.losses.mse(y, out)  # 计算每个样本的MSE
loss = tf.reduce_mean(loss)  # 平均MSE
loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.337504>

### 向量

向量是一种非常常见的数据载体，如在全连接层和卷积神经网络层中，偏置张量𝒃就使用向量来表示.

例: 2个输出节点的网络层, 创建长度为2的偏置向量**b**, 并累加在每个输出节点上


In [61]:
# z = wx , 模拟z
z = tf.random.normal([4, 2])
b = tf.zeros([2])
z = z + b  # 累加偏置向量
z

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[ 0.671354  ,  0.4674099 ],
       [-0.09950484,  0.994884  ],
       [ 0.7376844 , -0.44164726],
       [-0.17861803,  1.309803  ]], dtype=float32)>

In [6]:
# 创建一层网络层 张量𝑾和𝒃存储在类的内部，由类自动创建和管理
fc = layers.Dense(3)  # 创建一层Wx + b, 输出节点为3个
# 通过 build 函数创建 W,b 张量，输入节点为 4
fc.build(input_shape=(2, 4))
fc.bias  # 查看偏置向量
# 类型为Variable(待优化), 向量的值初始化为0

<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

### 矩阵

例如全连接层的批量输入张量$X$的形状为$[b, d_{in}]$, b表示输入样本的个数(Batch size), $d_{in}$表示输入特征的长度.


In [4]:
# 线性变换网络层, 激活函数为空
X = tf.random.normal([2, 4])  # 2个样本, 特征长度4
w = tf.ones([4, 3])
b = tf.zeros([3])

out = X@w + b
out

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-3.8082771, -3.8082771, -3.8082771],
       [-3.398396 , -3.398396 , -3.398396 ]], dtype=float32)>

X 和 w张量都是矩阵. 。一般地，$\sigma(X@W + b)$网络层称为`全连接层`，在 TensorFlow 中可以通过`Dense`类直接实现，特别地，当激活函数𝜎为空时，全连接层也称为`线性层`。

In [7]:
fc = layers.Dense(3) # 定义全连接层的输出节点为 3
fc.build(input_shape=(2,4)) # 定义全连接层的输入节点为 4
fc.kernel # 查看权值矩阵

<tf.Variable 'kernel:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.49846172, -0.03164536, -0.48681894],
       [ 0.00710827, -0.58150756, -0.29590786],
       [-0.22005463, -0.07923687, -0.16563272],
       [ 0.85872686,  0.77962816,  0.8142351 ]], dtype=float32)>

### 三维张量
三维张量的一个典型应用是表示`序列信号`, 格式是:
$$X = [b, sequence \; len, feature \; len]$$

其中𝑏表示序列信号的数量，sequence len 表示序列信号在时间维度上的采样点数或步数，feature len 表示每个点的特征长度.

考虑自然语言处理(Natural Language Processing，简称 NLP)中句子的表示，如评价句子的是否为正面情绪的情感分类任务网络。为了能够方便字符串被神经网络处理，一般将单词通过嵌入层(Embedding Layer)编码为固定长度的向量，比如“a”编码为某个长度 3 的向量，那么 2 个等长(单词数量为 5)的句子序列可以表示为 shape 为\[2,5,3\]的 3 维张量，其中 2 示句子个数，5 表示单词数量，3 表示单词向量的长度

![](./情感分类网络.png)

In [21]:
# imdb 电影评价数据集
(X_train, y_train), *_ = datasets.imdb.load_data(num_words=10000)

In [23]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 8255, 78, 228,

In [15]:
# 将句子填充 截断为等长的80个单词的句子
X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=80)

In [16]:
X_train.shape  # (句子个数, 每个句子单词数)每个单词用数字编码的方式表示 

(25000, 80)

In [18]:
# 创建词向量 embedding层
embedding = layers.Embedding(10000, 100)  # 每个单词都编码成长度100的向量

In [19]:
out = embedding(X_train)
out.shape  

TensorShape([25000, 80, 100])

### 四维张量